In [2]:
%load_ext jupyter_black

# nvidia cuda


``` bash
nvidia-smi --query-gpu=timestamp,name,pci.bus_id,driver_version,pstate,pcie.link.gen.max,pcie.link.gen.current,temperature.gpu,utilization.gpu,utilization.memory,memory.total,memory.free,memory.used --format=csv
nvidia-smi --query-gpu=name,temperature.gpu,utilization.gpu,utilization.memory,memory.total,memory.free,memory.used --format=csv -l 5

nvidia-smi -q -g 0 -d UTILIZATION -l
==============NVSMI LOG==============

Timestamp                                 : Sat Aug 13 18:07:02 2022
Driver Version                            : 516.59
CUDA Version                              : 11.7

Attached GPUs                             : 1
GPU 00000000:01:00.0
    Utilization
        Gpu                               : 1 %
        Memory                            : 1 %
        Encoder                           : 0 %
        Decoder                           : 0 %
    GPU Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 6 %
        Min                               : 0 %
        Avg                               : 0 %
    Memory Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 1 %
        Min                               : 0 %
        Avg                               : 0 %
    ENC Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 0 %
        Min                               : 0 %
        Avg                               : 0 %
    DEC Utilization Samples
        Duration                          : 98.31 sec
        Number of Samples                 : 99
        Max                               : 0 %
        Min                               : 0 %
        Avg                               : 0 %
```

In [2]:
import distributed
from dask_cuda import LocalCUDACluster


def close():
    """
    for convenience if a cluster/client is already running
    this should close them to prevent starting multiple clusters
    on the same port.
    """
    try:
        client.close(), cluster.close()
    except:
        ...


close()
cluster = LocalCUDACluster()
client = distributed.Client(cluster)

client

2022-08-14 01:14:43,380 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 15.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39945,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 15.57 GiB
Comm: tcp://127.0.0.1:44147,Total threads: 1
Dashboard: http://127.0.0.1:33599/status,Memory: 15.57 GiB
Nanny: tcp://127.0.0.1:43169,


In [3]:
import os
from pathlib import Path
import dask.dataframe as dd
import cudf

# creating a Path object from the absolute path to our data folder
ps_data = Path(os.path.abspath("probsevere-data"))
all_files = tuple(ps_data.rglob("*.pq"))
ps_data

PosixPath('/workspaces/griblib/notebooks/probsevere/probsevere-data')

In [8]:
ddf: dd.DataFrame = dd.read_parquet(all_files)
ddf.compute()

,MUCAPE,MLCAPE,MLCIN,EBSHEAR,SRH01KM,MEANWIND_1-3kmAGL,MESH,VIL_DENSITY,FLASH_RATE,FLASH_DENSITY,...,SIZE,MOTION_EAST,MOTION_SOUTH,PS,ID,valid_time,minx,miny,maxx,maxy
0,1524.0,1270.0,0.0,51.400002,36.0,17.900000,0.92,2.08,36.0,1.25,...,201.0,6.493,6.203,88.0,136524.0,2022-03-01 00:00:29,-79.209999,30.120001,-79.070000,30.280001
0,61.0,74.0,0.0,11.100000,225.0,46.000000,0.00,0.45,0.0,0.00,...,167.0,6.693,2.775,0.0,137200.0,2022-03-01 10:42:34,-122.099998,45.380001,-121.949997,45.509998
0,74.0,76.0,0.0,12.200000,227.0,45.599998,0.00,0.45,0.0,0.00,...,259.0,-0.263,-0.083,0.0,137200.0,2022-03-01 10:40:35,-122.169998,45.330002,-121.949997,45.500000
0,990.0,642.0,0.0,32.299999,27.0,13.300000,0.10,0.70,0.0,0.00,...,76.0,4.900,-0.477,2.0,137196.0,2022-03-01 10:38:37,-78.800003,26.600000,-78.709999,26.709999
0,990.0,642.0,0.0,32.299999,27.0,13.300000,0.11,0.72,0.0,0.00,...,76.0,-2.872,38.986,2.0,137196.0,2022-03-01 10:36:32,-78.800003,26.600000,-78.709999,26.709999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,4164.0,3341.0,-4.0,16.500000,16.0,10.200000,0.20,0.95,0.0,0.00,...,59.0,-4.459,-3.133,5.0,614114.0,2022-08-04 20:46:42,-81.839996,27.520000,-81.760002,27.600000
406,3612.0,2959.0,-1.0,7.700000,9.0,11.300000,0.14,0.52,0.0,0.00,...,64.0,-2.748,-4.769,2.0,614115.0,2022-08-04 20:46:42,-78.449997,27.160000,-78.370003,27.240000
407,4236.0,3416.0,-3.0,11.300000,26.0,13.000000,0.00,0.25,0.0,0.00,...,41.0,-4.422,0.006,2.0,614116.0,2022-08-04 20:46:42,-80.959999,25.480000,-80.830002,25.530001
408,2916.0,2220.0,-9.0,9.900000,46.0,13.500000,0.00,0.30,0.0,0.00,...,48.0,-1.091,-3.992,1.0,614117.0,2022-08-04 20:46:42,-78.660004,24.799999,-78.589996,24.889999


In [5]:
# the parquet files we're create using dask.dataframe
gdf: cudf.DataFrame = cudf.read_parquet(tuple(ps_data.rglob("*.pq")))
gdf[["ID","FLASH_RATE","PS"]] = gdf[["ID","FLASH_RATE","PS"]].astype(int)
df = gdf.set_index(["valid_time", "ID"]).sort_index().to_pandas()

del gdf  # delete the dataframe from gpu memory after it has been loaded to ram
df

MUCAPE  MLCAPE  MLCIN    EBSHEAR  SRH01KM  \
valid_time          ID                                                  
2022-03-01 00:00:29 136524  1524.0  1270.0    0.0  51.400002     36.0   
                    136565   756.0   433.0    0.0  39.200001     82.0   
                    136592   384.0   112.0   -5.0  37.900002     80.0   
                    136648   551.0   422.0   -4.0  33.299999     24.0   
                    136649    10.0     0.0 -999.0   6.400000    170.0   
...                            ...     ...    ...        ...      ...   
2022-08-04 23:58:40 617613  1649.0  1309.0  -13.0  17.000000      9.0   
                    617614  1362.0   947.0  -17.0  21.400000      3.0   
                    617615  2883.0  1545.0  -18.0  21.400000     37.0   
                    617616  3112.0  2861.0   -3.0  14.900000      9.0   
                    617617  2904.0  1940.0   -6.0   9.100000     42.0   

                            MEANWIND_1-3kmAGL  MESH  VIL_DENSITY  FLASH_RATE  \
valid_time          ID                                                         
2022-03-01 00:00:29 136524          17.900000  0.92         2.08          36   
                    136565          16.200001  0.11         0.68           0   
                    136592           7.300000  0.00         0.78           0   
                    136648          18.000000  0.08         0.94           0   
                    136649          39.000000  0.00         0.27           0   
...                                       ...   ...          ...         ...   
2022-08-04 23:58:40 617613           6.900000  0.00         0.05           0   
                    617614           9.300000  0.04         0.56           0   
                    617615           8.000000  0.38         0.62           0   
                    617616           7.300000  0.09         0.24           0   
                    617617          12.600000  0.00         0.24           0   

                            FLASH_DENSITY  ...  CAPE_M10M30  LJA   SIZE  \
valid_time          ID                     ...                            
2022-03-01 00:00:29 136524           1.25  ...        487.0  1.8  201.0   
                    136565           0.00  ...         77.0  0.0   55.0   
                    136592           0.00  ...        131.0  0.0  271.0   
                    136648           0.02  ...         60.0  0.0   83.0   
                    136649           0.00  ...          0.0  0.0  119.0   
...                                   ...  ...          ...  ...    ...   
2022-08-04 23:58:40 617613           0.00  ...        349.0  0.0   81.0   
                    617614           0.00  ...        373.0  0.0   55.0   
                    617615           0.00  ...        654.0  0.0  417.0   
                    617616           0.00  ...        628.0  0.0   45.0   
                    617617           0.00  ...        579.0  0.0   60.0   

                            MOTION_EAST  MOTION_SOUTH    PS        minx  \
valid_time          ID                                                    
2022-03-01 00:00:29 136524        6.493         6.203  88.0  -79.209999   
                    136565        8.784         5.193   4.0  -80.320000   
                    136592        5.375         5.236   2.0  -80.949997   
                    136648        6.686         5.148   2.0  -81.129997   
                    136649        5.171         4.775   0.0 -123.760002   
...                                 ...           ...   ...         ...   
2022-08-04 23:58:40 617613        2.746        -5.033   1.0  -82.599998   
                    617614       -1.828        -6.893   2.0 -108.760002   
                    617615       -2.596         0.509   9.0 -108.230003   
                    617616       -1.643         3.410   3.0  -93.430000   
                    617617        2.220        -2.021   1.0  -78.650002   

                                 miny        maxx       maxy  
valid_time          ID

In [53]:
from enum import Enum
class Cape(str,Enum):
    mixed_layer="MLCAPE"
class Motion(str,Enum):
    east="MOTION_EAST"
    south="MOTION_SOUTH"
    # cape=Cape
    # ml    
# 'MUCAPE', 'MLCAPE', 'MLCIN', 'EBSHEAR', 'SRH01KM', 'MEANWIND_1-3kmAGL',
#        'MESH', 'VIL_DENSITY', 'FLASH_RATE', 'FLASH_DENSITY', 'MAXLLAZ',
#        'P98LLAZ', 'P98MLAZ', 'WETBULB_0C_HGT', 'PWAT', 'CAPE_M10M30', 'LJA',
#        'SIZE', 'MOTION_EAST', 'MOTION_SOUTH', 'PS', 'minx', 'miny', 'maxx',
#        'maxy'
class columns(str,Enum):
    motion_east="MOTION_EAST"
    motion_south="MOTION_SOUTH"

df[[Motion.east,Motion.south, Cape.mixed_layer]]

MOTION_EAST  MOTION_SOUTH  MLCAPE
valid_time          ID                                       
2022-03-01 00:00:29 136524        6.493         6.203  1270.0
                    136565        8.784         5.193   433.0
                    136592        5.375         5.236   112.0
                    136648        6.686         5.148   422.0
                    136649        5.171         4.775     0.0
...                                 ...           ...     ...
2022-08-04 23:58:40 617613        2.746        -5.033  1309.0
                    617614       -1.828        -6.893   947.0
                    617615       -2.596         0.509  1545.0
                    617616       -1.643         3.410  2861.0
                    617617        2.220        -2.021  1940.0

[11105025 rows x 3 columns]

In [11]:
df.columns

Index(['MUCAPE', 'MLCAPE', 'MLCIN', 'EBSHEAR', 'SRH01KM', 'MEANWIND_1-3kmAGL',
       'MESH', 'VIL_DENSITY', 'FLASH_RATE', 'FLASH_DENSITY', 'MAXLLAZ',
       'P98LLAZ', 'P98MLAZ', 'WETBULB_0C_HGT', 'PWAT', 'CAPE_M10M30', 'LJA',
       'SIZE', 'MOTION_EAST', 'MOTION_SOUTH', 'PS', 'minx', 'miny', 'maxx',
       'maxy'],
      dtype='object')

In [11]:
df["ID"] = df["ID"].astype(int)
df.set_index(["valid_time", "ID"]).sort_index()

: 

: 

In [ ]:
df["ID"] = df["ID"].astype(int)
df.set_index(["valid_time", "ID"])


In [ ]:
abv20.compute()


In [ ]:
rows, cols = df.set_index(["valid_time", "ID"]).shape
rows * cols

In [ ]:
ddf2 = read_parquet()
# ids = ddf2.loc[(ddf2["PS"] > 20), "ID"]
# mask = ddf2["ID"] == ids
ddf2.loc[ddf2.groupby('ID').agg({"PS":"max"}) > 20,:].compute()